<a href="https://colab.research.google.com/github/Lucas-Ellenberger/CSE144/blob/main/stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import sys
import os

In [2]:
# make sure you're logged in with `huggingface-cli login`
from huggingface_hub import login

login()

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
!pip install torch
!pip install diffusers
!pip install accelerate
!pip install datasets
!pip install datasets[vision]

In [ ]:
import torch
from torch import autocast
import accelerate
from diffusers import StableDiffusionPipeline

In [ ]:
from diffusers import DDPMPipeline
from accelerate import Accelerator
from datasets import load_dataset

In [43]:
!pip install datasets
!pip install datasets[vision]
from datasets import load_dataset
dataset = load_dataset("AI4Math/MathVista")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [46]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

In [47]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# conditional image captioning
text = "a photography of"
inputs = processor(raw_image, text, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

# unconditional image captioning
inputs = processor(raw_image, return_tensors="pt")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))


/home/lucas_ellenberger/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a photography of a woman and her dog on the beach
woman sitting on the beach with her dog and a cell phone


In [44]:
# print the first example on the testmini set
print(dataset["testmini"][0])
print(dataset["testmini"][0]['pid']) # print the problem id
print(dataset["testmini"][0]['question']) # print the question text
print(dataset["testmini"][0]['query']) # print the query text
print(dataset["testmini"][0]['image']) # print the image path
print(dataset["testmini"][0]['answer']) # print the answer
dataset["testmini"][0]['decoded_image'] # display the image

# print the first example on the test set
print(dataset["test"][0])

{'pid': '1', 'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?", 'image': 'images/1.jpg', 'decoded_image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1514x720 at 0x7F2BADC14130>, 'choices': None, 'unit': None, 'precision': 1.0

In [67]:
# Uncomment if you want to see a sample image from the dataset
# categories = set()
# count = 0
decriptions = []
# for i in range(len(dataset['testmini'])):
for i in range(5):
    inputs = processor(dataset["testmini"][i]['decoded_image'], return_tensors="pt")
    out = model.generate(**inputs)
    descriptions.append(processor.decode(out[0], skip_special_tokens=True))

print(descriptions)
addition = {
    'description': descriptions
}
# print("count: ", count)
# print("len(testmini): ", len(dataset['testmini']))
# print(count / len(dataset['testmini']))
# print(categories)
# print(len(dataset['test']))
# i = 8
# print(dataset['testmini'][i]['metadata']['category'])
# print(dataset["testmini"][i]['query']) # print the question text
# unconditional image captioning
# inputs = processor(dataset["testmini"][i]['decoded_image'], return_tensors="pt")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))
# dataset['testmini'][i]['decoded_image']

/home/lucas_ellenberger/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['diagram of a spring force does negative work, decreasing speed and kinetic energy', 'there is a bowl of flour and eggs on a cutting board', 'a drawing of a triangle with three intersecting sides', 'there are three different colored objects on a white surface', 'a picture of a diagram of a right triangle with a right angle', 'a black and white photo of a number of letters on a line', 'a close up of a clock with a purple background and a white face', 'there are many different types of toys that are on the table', 'a picture of a box with a letter a and a box with a letter b', 'a close up of a cell phone with a number of letters', 'diagram of a spring force does negative work, decreasing speed and kinetic energy', 'there is a bowl of flour and eggs on a cutting board', 'a drawing of a triangle with three intersecting sides', 'there are three different colored objects on a white surface', 'a picture of a diagram of a right triangle with a right angle']


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
pipe = StableDiffusionPipeline.from_pretrained(
	"CompVis/stable-diffusion-v1-4",
	use_auth_token=True
).to(device)

prompt = "a photo of an astronaut riding a horse on mars"
with autocast(device):
    image = pipe(prompt)["sample"][0]

image.save("astronaut_rides_horse.png")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: User specified an unsupported autocast device_type 'cpu'